> In this notebook, all the values before sowing period and 1 FN after harvest are made zeroes.

# Import Modules and Data

In [1]:
from glob import glob
import geopandas as gp
import numpy as np
import pandas as pd
from copy import deepcopy
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from IPython.display import display
import random

pd.options.display.max_rows = 100

import warnings
warnings.filterwarnings("ignore")

In [5]:
# Importing all the required files
wheat_train_val = pd.read_csv(r'data_files\data_share\preprocessed_wheat.csv')
mustard_train_val = pd.read_csv(r'data_files\data_share\preprocessed_mustard.csv')

wheat_test = pd.read_csv(r'data_files/data_share/preprocessed_new_wheat.csv')
mustard_test = pd.read_csv(r'data_files/data_share/preprocessed_new_mustard.csv')

# Dropping the unnecessary features as of now
wheat_train_val = wheat_train_val.drop(['latitude', 'longitude', 'state_name', 'district', 'taluka_name', 'sowing_year', 'harvest_year'], axis=1)
mustard_train_val = mustard_train_val.drop(['latitude', 'longitude', 'state_name', 'district', 'taluka_name', 'sowing_year', 'harvest_year'], axis=1)

In [6]:
# Not using the combinations, where the count is just 1

display(pd.crosstab(wheat_train_val['sowing_period'], wheat_train_val['harvest_period'], margins=True))
display(pd.crosstab(mustard_train_val['sowing_period'], mustard_train_val['harvest_period'], margins=True))

harvest_period,feb_1f,feb_2f,jan_2f,mar_1f,mar_2f,All
sowing_period,,,,,,
dec_1f,0,0,0,2033,3112,5145
dec_2f,0,0,0,19,166,185
nov_1f,118,1433,1,1954,181,3687
nov_2f,10,1082,0,4505,1761,7358
oct_2f,59,68,1,28,12,168
All,187,2583,2,8539,5232,16543


harvest_period,feb_1f,feb_2f,jan_2f,mar_1f,mar_2f,All
sowing_period,,,,,,
nov_1f,34,296,1,375,18,724
nov_2f,2,4,0,13,1,20
oct_2f,148,319,11,60,3,541
All,184,619,12,448,22,1285


In [7]:
#Removing 'Wheat_oct_2f - Wheat_jan_2f' and 'Wheat_nov_1f - Wheat_jan_2f' combinations as they are just 1 in count and cant be used for stratification

wheat_train_val = wheat_train_val[(wheat_train_val.sowing_period != 'nov_1f') | (wheat_train_val.harvest_period != 'jan_2f')]
wheat_train_val = wheat_train_val[(wheat_train_val.sowing_period != 'oct_2f') | (wheat_train_val.harvest_period != 'jan_2f')]                                 

#Removing 'Mustard_nov_1f - Mustard_jan_2f' and 'Mustard_nov_2f - Mustard_mar_2f' combinations as they are just 1 in count and cant be used for stratification
mustard_train_val = mustard_train_val[(mustard_train_val.sowing_period != 'nov_1f') | (mustard_train_val.harvest_period != 'jan_2f')]
mustard_train_val = mustard_train_val[(mustard_train_val.sowing_period != 'nov_2f') | (mustard_train_val.harvest_period != 'mar_2f')]          

# Creating train_val and test data sets
train_val = pd.concat([wheat_train_val, mustard_train_val], ignore_index=True)
test = pd.concat([wheat_test, mustard_test], ignore_index=True)

# Adding zeroes before Sowing_Period and after harvest

In [8]:
# Adding zeroes before SP and after harvest

def zeroes_bf_sp_af_hr(row):
    values = deepcopy(row)
    sowing_period = values['sowing_period'][-6:]
    harvest = values['harvest_period'][-6:]
    
    if (sowing_period != 'oct_2f'):
        sowing_period_idx = values.index.get_loc(sowing_period)
        strt_fn_bf_sp = values.index[sowing_period_idx-1]
        values.loc[:strt_fn_bf_sp] = len(values.loc[:strt_fn_bf_sp]) * [0]
        
    if (harvest not in {'mar_2f', 'apr_1f'}):    
        harvest_idx = values.index.get_loc(harvest)
        strt_fn_af_hr = values.index[harvest_idx+2]
        values.loc[strt_fn_af_hr:'apr_1f'] = len(values.loc[strt_fn_af_hr:'apr_1f']) * [0]
        
    return values

train_val = train_val.apply(zeroes_bf_sp_af_hr, axis=1)
test = test.apply(zeroes_bf_sp_af_hr, axis=1)

# Train_val Split

In [9]:
train, val = train_test_split(train_val, test_size=0.2, 
                                   stratify=train_val[['sowing_period', 'harvest_period']], random_state=0)

display(train.shape, val.shape, train.crop_name.value_counts(normalize=True), val.crop_name.value_counts(normalize=True))

(14259, 15)

(3565, 15)

Wheat      0.928256
Mustard    0.071744
Name: crop_name, dtype: float64

Wheat      0.927069
Mustard    0.072931
Name: crop_name, dtype: float64

# Label Encoding

In [10]:
for df in train, val, test:
    df['crop_name'] = df.crop_name.apply(lambda crop: 1 if crop == 'Wheat' else 0)
    df.drop(['sowing_period', 'harvest_period'], axis=1, inplace=True)
    
train.head(3)

,oct_2f,nov_1f,nov_2f,dec_1f,dec_2f,jan_1f,jan_2f,feb_1f,feb_2f,mar_1f,mar_2f,apr_1f,crop_name
9846,0,0,114,122,162,178.0,180,193,190,181,155,121,1
16127,0,0,0,117,137,172.0,172,186,181,177,145,132,1
3135,0,114,123,170,190,192.0,192,191,194,155,121,0,1


# File Export

In [11]:
train.to_csv(r'data_files\data_share\train-2.csv', index=False)
test.to_csv(r'data_files\data_share\test.csv', index=False)
val.to_csv(r'data_files\data_share\val-2.csv', index=False)